# **Homework 4**

In [1]:
#Step 1: Install Dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark

#Step 2: Add environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

#Step 3: Initialize Pyspark
import findspark
findspark.init()

In [2]:
#creating spark context
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [8]:
df = spark.read.format("csv").option("header", "true").load("kroger_stock.csv")

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Kroger Stock data.

#### Use the kroger_stock.csv file to Answer and complete the  tasks below!

#### What are the column names?

In [9]:
columns = df.columns
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


#### What does the Schema look like?

In [10]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



#### Print out the first 5 columns.

In [11]:
df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [12]:
df.describe()

DataFrame[summary: string, Date: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [30]:
hv_ratio_df = df.withColumn("HV Ratio", df["High"] / df["Volume"])
hv_ratio_df.show(5)

+----------+------------------+---------+---------+------------------+---------+------------------+----+--------------------+
|      Date|              Open|     High|      Low|             Close|   Volume|         Adj Close|Year|            HV Ratio|
+----------+------------------+---------+---------+------------------+---------+------------------+----+--------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|1.26688E7|52.619234999999996|2012|4.819714653321546E-6|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996|9593300.0|         52.078475|2012|6.290848613094555E-6|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|1.27682E7|         51.825539|2012|4.669412994783916E-6|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0|8069400.0|          51.45922|2012|7.367338463826307E-6|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18|6679300.0|51.616215000000004|2012|8.915604778943

#### What day had the Peak High in Price?

In [16]:
sorted_df = df.orderBy(df["High"].desc())
highest_peak_day = sorted_df.first()
print(highest_peak_day["Date"])

2015-01-13


#### What is the mean of the Close column?

In [17]:
mean_close = df.agg({"Close": "mean"}).collect()[0][0]
print(mean_close)

72.38844998012726


#### What is the max and min of the Volume column?

In [19]:
df = df.withColumn("Volume", df["Volume"].cast("double"))
max_volume = df.agg({"Volume": "max"}).collect()[0][0]
min_volume = df.agg({"Volume": "min"}).collect()[0][0]
print("The max Volume is", max_volume, "and the min Volume is", min_volume)

The max Volume is 80898100.0 and the min Volume is 2094900.0


#### How many days was the Close lower than 60 dollars?

In [20]:
filtered_df = df.filter(df.Close < 60)
num_days = filtered_df.count()
print(num_days)

81


#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [21]:
num_high = df.filter(df.High > 80).count()
total_days = df.count()
percentage_high = (num_high / total_days) * 100
print(percentage_high)

8.426073131955485


#### What is the max High per year?

In [22]:
df = df.withColumn("Year", df.Date.substr(1, 4))
max_high_per_year = df.groupBy("Year").agg({"High": "max"})
max_high_per_year.show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [27]:
df_with_month = df.withColumn('Month', df['Date'].substr(6, 2))
avg_close_by_month = df_with_month.groupBy('Month').agg({'Close': 'mean'}).orderBy('Month')
avg_close_by_month.show()


+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|   01|71.44801958415842|
|   02|  71.306804443299|
|   03|71.77794377570092|
|   04|72.97361900952382|
|   05|72.30971688679247|
|   06| 72.4953774245283|
|   07|74.43971943925233|
|   08|73.02981855454546|
|   09|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

